ELEC6910X Advanced Topics in AI and Healthcare

# Assignment 1

***Holy Lovenia - 20814158***

------

In [1]:
from collections import OrderedDict
from PIL import Image
from pytorch3dunet.unet3d import losses
from torch import nn
from torch.autograd import Variable
from torch.nn import init
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomVerticalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)
from transformers import AutoConfig, AutoFeatureExtractor, AutoModelForImageClassification, TrainingArguments, Trainer, EarlyStoppingCallback
from tqdm import tqdm

import datasets
import h5py
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import surface_distance.metrics as sf
import torch
import torch.nn.functional as F
import torch.optim as optim
import torchio as tio
import torchmetrics
import torchvision

In [2]:
RANDOM_SEED = 42

def set_all_seeds(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.backends.cudnn.deterministic = True

set_all_seeds(RANDOM_SEED)

## Problem 2

In [3]:
model_name_or_path = "microsoft/resnet-152"
# model_name_or_path = "gianlab/swin-tiny-patch4-window7-224-finetuned-skin-cancer"
# model_name_or_path = "google/vit-base-patch16-224-in21k"

In [4]:
metric = datasets.load_metric("accuracy")

/tmp/ipykernel_29746/3770023701.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("accuracy")


In [5]:
def problem2_load_dataset(manifest_path, img_dir_path, img_column_name="image_train", label_column_name="label_train"):
    manifest_df = pd.read_csv(manifest_path)
    manifest_df[img_column_name] = img_dir_path + manifest_df[img_column_name].astype(str)
    manifest_df = manifest_df.rename(columns={img_column_name: "image", label_column_name: "label"})
    dataset = datasets.Dataset.from_pandas(manifest_df)
    dataset = dataset.cast_column("image", datasets.Image(decode=True))
    dataset = dataset.class_encode_column("label")
    return dataset

In [6]:
def ham10000_load_dataset(manifest_path, img_dir_paths, img_column_name="image_id", label_column_name="dx"):
    manifest_df = pd.read_csv(manifest_path)
    for i in range(len(manifest_df)):
        for img_dir_path in img_dir_paths:
            img_path = os.path.join(img_dir_path, f"{manifest_df[img_column_name][i]}.jpg")
            if os.path.isfile(img_path):
                manifest_df[img_column_name][i] = img_path
    manifest_df = manifest_df.drop(manifest_df[~manifest_df[img_column_name].str.contains(".jpg")].index)
    manifest_df = manifest_df.rename(columns={img_column_name: "image", label_column_name: "label"})
    manifest_df = manifest_df.replace({
        "mel": 0,
        "nv": 1,
        "bcc": 2,
        "akiec": 3,
        "bkl": 4,
        "df": 5,
        "vasc": 6,
    })
    dataset = datasets.Dataset.from_pandas(manifest_df)
    dataset = dataset.cast_column("image", datasets.Image(decode=True))
    dataset = dataset.class_encode_column("label")
    return dataset

In [7]:
raw_datasets = datasets.DatasetDict()
raw_datasets["train"] = problem2_load_dataset("./data/problem2_datas/annotation/train.csv", "./data/problem2_datas/images/")
# train_val_dataset = problem2_load_dataset("./data/problem2_datas/annotation/train.csv", "./data/problem2_datas/images/").train_test_split(test_size=0.2)
# raw_datasets["train"] = train_val_dataset["train"]
# raw_datasets["valid"] = train_val_dataset["test"]
raw_datasets["test"] = problem2_load_dataset("./data/problem2_datas/annotation/test.csv", "./data/problem2_datas/images/")

Stringifying the column:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting to class labels:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Stringifying the column:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting to class labels:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [8]:
pretrain_datasets = datasets.DatasetDict()
pretrain_train_val_dataset = ham10000_load_dataset(
    "./data/HAM10000/HAM10000_metadata.csv",
    [
        "./data/HAM10000/ham10000_images_part_1",
        "./data/HAM10000/HAM10000_images_part_1",
        "./data/HAM10000/ham10000_images_part_2",
        "./data/HAM10000/HAM10000_images_part_2"
    ]).train_test_split(test_size=0.2)
pretrain_datasets["train"] = pretrain_train_val_dataset["train"]
pretrain_datasets["valid"] = pretrain_train_val_dataset["test"]

/tmp/ipykernel_29746/4003405763.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  manifest_df[img_column_name][i] = img_path


Stringifying the column:   0%|          | 0/11 [00:00<?, ?ba/s]

Casting to class labels:   0%|          | 0/11 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/2 [00:00<?, ?ba/s]

In [9]:
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name_or_path)
model = AutoModelForImageClassification.from_pretrained(model_name_or_path)

In [10]:
normalize = Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
train_transforms = Compose(
        [
#             RandomResizedCrop(feature_extractor.size),
            Resize(feature_extractor.size),
            CenterCrop(feature_extractor.size),
            RandomHorizontalFlip(),
            RandomVerticalFlip(),
            ToTensor(),
            normalize,
        ]
    )

eval_transforms = Compose(
        [
            Resize(feature_extractor.size),
            CenterCrop(feature_extractor.size),
            ToTensor(),
            normalize,
        ]
    )

def preprocess_train(example_batch):
    """Apply train_transforms across a batch."""
    example_batch["pixel_values"] = [
        train_transforms(image.convert("RGB")) for image in example_batch["image"]
    ]
    return example_batch

def preprocess_eval(example_batch):
    """Apply eval_transforms across a batch."""
    example_batch["pixel_values"] = [
        eval_transforms(image.convert("RGB")) for image in example_batch["image"]
    ]
    return example_batch

In [11]:
proc_datasets = raw_datasets
proc_datasets["train"].set_transform(preprocess_train)
# proc_datasets["valid"].set_transform(preprocess_eval)
proc_datasets["test"].set_transform(preprocess_eval)

In [12]:
proc_pretrain_datasets = pretrain_datasets
proc_pretrain_datasets["train"].set_transform(preprocess_train)
proc_pretrain_datasets["valid"].set_transform(preprocess_eval)

In [13]:
def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [14]:
def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

#### Pretrain

In [15]:
args = TrainingArguments(
    f"save/problem2/pretrain__{model_name_or_path}__1e-4__weighted",
    remove_unused_columns=False,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=200,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    gradient_checkpointing=True,
    save_total_limit=1,
    seed=RANDOM_SEED,
    data_seed=RANDOM_SEED,
    dataloader_num_workers=4,
    lr_scheduler_type="cosine",
)

In [16]:
class Problem2PretrainingTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels").to("cuda")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits").to("cuda")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([0.11, 0.68, 0.05, 0.03, 0.11, 0.01, 0.01] + [0 for i in range(993)])).to("cuda")
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [17]:
trainer = Problem2PretrainingTrainer(
    model,
    args,
    train_dataset=proc_pretrain_datasets["train"],
    eval_dataset=proc_pretrain_datasets["valid"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=10)],
)

In [18]:
train_results = trainer.train()
# rest is optional but nice to have
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

/home/holy/anaconda3/envs/health/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1600
  Num Epochs = 200
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 4
  Total optimization steps = 2400


Epoch,Training Loss,Validation Loss,Accuracy
0,7.544700,7.498845,0.000000
1,7.885700,7.345616,0.000000
2,7.692600,7.076983,0.002500
3,7.527200,6.792136,0.002500
4,6.659300,6.471617,0.005000
5,6.679300,6.080491,0.007500
6,6.337000,5.600641,0.057500
7,5.877100,4.792562,0.187500
8,5.287800,3.794149,0.360000
9,3.437600,2.845498,0.522500


***** Running Evaluation *****
  Num examples = 400
  Batch size = 32
Saving model checkpoint to save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/checkpoint-12
Configuration saved in save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/checkpoint-12/config.json
Model weights saved in save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/checkpoint-12/pytorch_model.bin
Feature extractor saved in save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/checkpoint-12/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 400
  Batch size = 32
Saving model checkpoint to save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/checkpoint-24
Configuration saved in save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/checkpoint-24/config.json
Model weights saved in save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/checkpoint-24/pytorch_model.bin
Feature extractor saved in save/problem2/pretrain__microsoft/resnet-152__1e-4

Deleting older checkpoint [save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/checkpoint-144] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 400
  Batch size = 32
Saving model checkpoint to save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/checkpoint-168
Configuration saved in save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/checkpoint-168/config.json
Model weights saved in save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/checkpoint-168/pytorch_model.bin
Feature extractor saved in save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/checkpoint-168/preprocessor_config.json
Deleting older checkpoint [save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/checkpoint-156] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 400
  Batch size = 32
Saving model checkpoint to save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/checkpoint-180
Configuration saved in save/prob

Configuration saved in save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/checkpoint-312/config.json
Model weights saved in save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/checkpoint-312/pytorch_model.bin
Feature extractor saved in save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/checkpoint-312/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 400
  Batch size = 32
Saving model checkpoint to save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/checkpoint-324
Configuration saved in save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/checkpoint-324/config.json
Model weights saved in save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/checkpoint-324/pytorch_model.bin
Feature extractor saved in save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/checkpoint-324/preprocessor_config.json
Deleting older checkpoint [save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/checkpoint-312] due to args

Deleting older checkpoint [save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/checkpoint-444] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 400
  Batch size = 32
Saving model checkpoint to save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/checkpoint-468
Configuration saved in save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/checkpoint-468/config.json
Model weights saved in save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/checkpoint-468/pytorch_model.bin
Feature extractor saved in save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/checkpoint-468/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 400
  Batch size = 32
Saving model checkpoint to save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/checkpoint-480
Configuration saved in save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/checkpoint-480/config.json
Model weights saved in save/problem2/pretrain__microso

Configuration saved in save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/checkpoint-612/config.json
Model weights saved in save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/checkpoint-612/pytorch_model.bin
Feature extractor saved in save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/checkpoint-612/preprocessor_config.json
Deleting older checkpoint [save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/checkpoint-600] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 400
  Batch size = 32
Saving model checkpoint to save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/checkpoint-624
Configuration saved in save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/checkpoint-624/config.json
Model weights saved in save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/checkpoint-624/pytorch_model.bin
Feature extractor saved in save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/checkpoint-624/preproc

***** train metrics *****
  epoch                    =        60.96
  total_flos               = 4941548852GF
  train_loss               =        1.254
  train_runtime            =   0:22:37.96
  train_samples_per_second =      235.647
  train_steps_per_second   =        1.767


In [19]:
metrics = trainer.evaluate(proc_datasets["test"])
# some nice to haves:
trainer.log_metrics("test", metrics)
trainer.save_metrics("test", metrics)

***** Running Evaluation *****
  Num examples = 99
  Batch size = 32


***** test metrics *****
  epoch                   =      60.96
  eval_accuracy           =      0.202
  eval_loss               =     8.0957
  eval_runtime            = 0:00:01.76
  eval_samples_per_second =     56.067
  eval_steps_per_second   =      2.265


In [20]:
metrics = trainer.evaluate(proc_pretrain_datasets["valid"])
# some nice to haves:
trainer.log_metrics("valid", metrics)
trainer.save_metrics("valid", metrics)

***** Running Evaluation *****
  Num examples = 400
  Batch size = 32


***** valid metrics *****
  epoch                   =      60.96
  eval_accuracy           =     0.8675
  eval_loss               =     0.5119
  eval_runtime            = 0:00:03.08
  eval_samples_per_second =    129.543
  eval_steps_per_second   =       4.21


#### Problem 2 dataset

In [21]:
model = AutoModelForImageClassification.from_pretrained(f"save/problem2/pretrain__{model_name_or_path}__1e-4__weighted")
# model = AutoModelForImageClassification.from_pretrained(model_name_or_path)

loading configuration file save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/config.json
Model config ResNetConfig {
  "_name_or_path": "save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted",
  "architectures": [
    "ResNetForImageClassification"
  ],
  "depths": [
    3,
    8,
    36,
    3
  ],
  "downsample_in_first_stage": false,
  "embedding_size": 64,
  "hidden_act": "relu",
  "hidden_sizes": [
    256,
    512,
    1024,
    2048
  ],
  "id2label": {
    "0": "tench, Tinca tinca",
    "1": "goldfish, Carassius auratus",
    "2": "great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias",
    "3": "tiger shark, Galeocerdo cuvieri",
    "4": "hammerhead, hammerhead shark",
    "5": "electric ray, crampfish, numbfish, torpedo",
    "6": "stingray",
    "7": "cock",
    "8": "hen",
    "9": "ostrich, Struthio camelus",
    "10": "brambling, Fringilla montifringilla",
    "11": "goldfinch, Carduelis carduelis",
    "12": "house finch, l

loading weights file save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted/pytorch_model.bin
All model checkpoint weights were used when initializing ResNetForImageClassification.

All the weights of ResNetForImageClassification were initialized from the model checkpoint at save/problem2/pretrain__microsoft/resnet-152__1e-4__weighted.
If your task is similar to the task the model of the checkpoint was trained on, you can already use ResNetForImageClassification for predictions without further training.


In [22]:
args = TrainingArguments(
    f"save/problem2/pretrained__1e-4__pretrained|{model_name_or_path}__1e-3",
#     f"save/problem2/{model_name_or_path}__5e-4__cwr",
    remove_unused_columns=False,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1e-3,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=200,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    gradient_checkpointing=True,
    save_total_limit=1,
    seed=RANDOM_SEED,
    data_seed=RANDOM_SEED,
    dataloader_num_workers=4,
    lr_scheduler_type="cosine",
    fp16=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [23]:
class Problem2Trainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels").to("cuda")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits").to("cuda")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([0.11, 0.68, 0.05, 0.03, 0.11, 0.01, 0.01] + [0 for i in range(993)])).to("cuda")
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [24]:
trainer = Problem2Trainer(
    model,
    args,
    train_dataset=proc_datasets["train"],
    eval_dataset=proc_datasets["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=20)],
)

Using cuda_amp half precision backend


In [25]:
train_results = trainer.train()
# rest is optional but nice to have
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

/home/holy/anaconda3/envs/health/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 305
  Num Epochs = 200
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 4
  Total optimization steps = 400


Epoch,Training Loss,Validation Loss,Accuracy
0,No log,9.464192,0.171717
1,No log,9.788841,0.171717
2,No log,9.715256,0.171717
3,No log,8.673621,0.171717
4,11.914200,8.048030,0.171717
5,11.914200,5.873352,0.181818
6,11.914200,3.418836,0.242424
7,11.914200,2.036694,0.565657
8,11.914200,1.223399,0.666667
9,5.847900,0.785580,0.737374


***** Running Evaluation *****
  Num examples = 99
  Batch size = 32
Saving model checkpoint to save/problem2/pretrained__1e-4__pretrained|microsoft/resnet-152__1e-3/checkpoint-2
Configuration saved in save/problem2/pretrained__1e-4__pretrained|microsoft/resnet-152__1e-3/checkpoint-2/config.json
Model weights saved in save/problem2/pretrained__1e-4__pretrained|microsoft/resnet-152__1e-3/checkpoint-2/pytorch_model.bin
Feature extractor saved in save/problem2/pretrained__1e-4__pretrained|microsoft/resnet-152__1e-3/checkpoint-2/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 99
  Batch size = 32
Saving model checkpoint to save/problem2/pretrained__1e-4__pretrained|microsoft/resnet-152__1e-3/checkpoint-4
Configuration saved in save/problem2/pretrained__1e-4__pretrained|microsoft/resnet-152__1e-3/checkpoint-4/config.json
Model weights saved in save/problem2/pretrained__1e-4__pretrained|microsoft/resnet-152__1e-3/checkpoint-4/pytorch_model.bin
Feature extractor saved

***** Running Evaluation *****
  Num examples = 99
  Batch size = 32
Saving model checkpoint to save/problem2/pretrained__1e-4__pretrained|microsoft/resnet-152__1e-3/checkpoint-26
Configuration saved in save/problem2/pretrained__1e-4__pretrained|microsoft/resnet-152__1e-3/checkpoint-26/config.json
Model weights saved in save/problem2/pretrained__1e-4__pretrained|microsoft/resnet-152__1e-3/checkpoint-26/pytorch_model.bin
Feature extractor saved in save/problem2/pretrained__1e-4__pretrained|microsoft/resnet-152__1e-3/checkpoint-26/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 99
  Batch size = 32
Saving model checkpoint to save/problem2/pretrained__1e-4__pretrained|microsoft/resnet-152__1e-3/checkpoint-28
Configuration saved in save/problem2/pretrained__1e-4__pretrained|microsoft/resnet-152__1e-3/checkpoint-28/config.json
Model weights saved in save/problem2/pretrained__1e-4__pretrained|microsoft/resnet-152__1e-3/checkpoint-28/pytorch_model.bin
Feature extracto

***** Running Evaluation *****
  Num examples = 99
  Batch size = 32
Saving model checkpoint to save/problem2/pretrained__1e-4__pretrained|microsoft/resnet-152__1e-3/checkpoint-50
Configuration saved in save/problem2/pretrained__1e-4__pretrained|microsoft/resnet-152__1e-3/checkpoint-50/config.json
Model weights saved in save/problem2/pretrained__1e-4__pretrained|microsoft/resnet-152__1e-3/checkpoint-50/pytorch_model.bin
Feature extractor saved in save/problem2/pretrained__1e-4__pretrained|microsoft/resnet-152__1e-3/checkpoint-50/preprocessor_config.json
Deleting older checkpoint [save/problem2/pretrained__1e-4__pretrained|microsoft/resnet-152__1e-3/checkpoint-48] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 99
  Batch size = 32
Saving model checkpoint to save/problem2/pretrained__1e-4__pretrained|microsoft/resnet-152__1e-3/checkpoint-52
Configuration saved in save/problem2/pretrained__1e-4__pretrained|microsoft/resnet-152__1e-3/checkpoint-52/config.json


Deleting older checkpoint [save/problem2/pretrained__1e-4__pretrained|microsoft/resnet-152__1e-3/checkpoint-70] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 99
  Batch size = 32
Saving model checkpoint to save/problem2/pretrained__1e-4__pretrained|microsoft/resnet-152__1e-3/checkpoint-74
Configuration saved in save/problem2/pretrained__1e-4__pretrained|microsoft/resnet-152__1e-3/checkpoint-74/config.json
Model weights saved in save/problem2/pretrained__1e-4__pretrained|microsoft/resnet-152__1e-3/checkpoint-74/pytorch_model.bin
Feature extractor saved in save/problem2/pretrained__1e-4__pretrained|microsoft/resnet-152__1e-3/checkpoint-74/preprocessor_config.json
Deleting older checkpoint [save/problem2/pretrained__1e-4__pretrained|microsoft/resnet-152__1e-3/checkpoint-72] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 99
  Batch size = 32
Saving model checkpoint to save/problem2/pretrained__1e-4__pretrained|microsoft/resnet-15

Model weights saved in save/problem2/pretrained__1e-4__pretrained|microsoft/resnet-152__1e-3/pytorch_model.bin
Feature extractor saved in save/problem2/pretrained__1e-4__pretrained|microsoft/resnet-152__1e-3/preprocessor_config.json


***** train metrics *****
  epoch                    =        46.8
  total_flos               = 725789987GF
  train_loss               =      1.9939
  train_runtime            =  0:05:40.24
  train_samples_per_second =     179.284
  train_steps_per_second   =       1.176


In [26]:
metrics = trainer.evaluate(proc_datasets["test"])
# some nice to haves:
trainer.log_metrics("test", metrics)
trainer.save_metrics("test", metrics)

***** Running Evaluation *****
  Num examples = 99
  Batch size = 32


***** test metrics *****
  epoch                   =       46.8
  eval_accuracy           =     0.8081
  eval_loss               =     0.2789
  eval_runtime            = 0:00:01.97
  eval_samples_per_second =     50.121
  eval_steps_per_second   =      2.025


In [27]:
# metrics = trainer.evaluate(proc_datasets["valid"])
# # some nice to haves:
# trainer.log_metrics("valid", metrics)
# trainer.save_metrics("valid", metrics)

In [28]:
metrics = trainer.evaluate(proc_datasets["train"])
# some nice to haves:
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)

***** Running Evaluation *****
  Num examples = 305
  Batch size = 32


***** train metrics *****
  epoch                   =       46.8
  eval_accuracy           =     0.9443
  eval_loss               =     0.0123
  eval_runtime            = 0:00:02.81
  eval_samples_per_second =     108.22
  eval_steps_per_second   =      3.548
